In [179]:
import pandas as pd
import json

In [180]:
standing_path = '../step_0/standings/standing_2024.json'
match_path = '../step_0/matchesAndH2H/match_2024.json'
with open(standing_path, 'r', encoding='utf-8') as f:
    standing_data = json.load(f)
with open(match_path, 'r', encoding='utf-8') as f:
    match_data = json.load(f)

In [181]:
total_standings = standing_data['standings'][0]['table']
home_standings = standing_data['standings'][1]['table']
away_standings = standing_data['standings'][2]['table']

total_df = pd.json_normalize(total_standings)
home_df = pd.json_normalize(home_standings)
away_df = pd.json_normalize(away_standings)

merged_df = total_df.merge(home_df, on='team.id', suffixes=('', '_home')).merge(away_df, on='team.id', suffixes=('', '_away'))

standing_df = merged_df[[
    'team.shortName', 'position', 'playedGames', 'form', 'won', 'draw', 'lost', 'points', 'goalsFor', 'goalsAgainst', 'goalDifference',
    'won_home', 'draw_home', 'lost_home', 'points_home', 'goalsFor_home', 'goalsAgainst_home', 'goalDifference_home',
    'won_away', 'draw_away', 'lost_away', 'points_away', 'goalsFor_away', 'goalsAgainst_away', 'goalDifference_away'
]]

standing_df.columns = [
    'Team', 'Pos', 'PG', 'Form', 'W', 'D', 'L', 'P', 'GF', 'GA', 'GD', 
    'HW', 'HD', 'HL', 'HP', 'HGF', 'HGA', 'HGD', 
    'AW', 'AD', 'AL', 'AP', 'AGF', 'AGA', 'AGD',
]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

standing_df.head()

,Team,Pos,PG,Form,W,D,L,P,GF,GA,GD,HW,HD,HL,HP,HGF,HGA,HGD,AW,AD,AL,AP,AGF,AGA,AGD
0,Liverpool,1,7,"W,W,W,L,W",6,0,1,18,13,2,11,2,0,1,6,5,1,4,4,0,0,12,8,1,7
1,Man City,2,7,"W,D,D,W,W",5,2,0,17,17,8,9,3,1,0,10,11,6,5,2,1,0,7,6,2,4
2,Arsenal,3,7,"W,W,D,W,D",5,2,0,17,15,6,9,3,1,0,10,10,4,6,2,1,0,7,5,2,3
3,Chelsea,4,7,"D,W,W,W,D",4,2,1,14,16,8,8,1,2,1,5,6,6,0,3,0,0,9,10,2,8
4,Aston Villa,5,7,"D,D,W,W,W",4,2,1,14,12,9,3,2,1,1,7,6,5,1,2,1,0,7,6,4,2


In [182]:
matches = match_data['matches']

match_list = []
for match in matches:
    match_list.append({
        'Match Id': match['id'],
        'Home Team': match['homeTeam']['shortName'],
        'Away Team': match['awayTeam']['shortName'],
        'Home Goal HT': match['score']['halfTime']['home'],
        'Away Goal HT': match['score']['halfTime']['away'],
        'Home Goal FT': match['score']['fullTime']['home'],
        'Away Goal FT': match['score']['fullTime']['away']
    })

match_df = pd.DataFrame(match_list)

match_df.head()

,Match Id,Home Team,Away Team,Home Goal HT,Away Goal HT,Home Goal FT,Away Goal FT
0,497410,Man United,Fulham,0.0,0.0,1.0,0.0
1,497411,Ipswich Town,Liverpool,0.0,0.0,0.0,2.0
2,497412,Arsenal,Wolverhampton,1.0,0.0,2.0,0.0
3,497413,Everton,Brighton Hove,0.0,1.0,0.0,3.0
4,497414,Newcastle,Southampton,1.0,0.0,1.0,0.0


In [183]:
h2h_list = []
for match in matches:
    h2h_data = match.get('head2head', {})
    h2h_matches = h2h_data.get('matches', [])
    
    for h2h_match in h2h_matches:
        h2h_list.append({
            'Match Id': match['id'],
            'H2H Match Id': h2h_match['id'],
            'Home Team': h2h_match['homeTeam']['shortName'],
            'Away Team': h2h_match['awayTeam']['shortName'],
            'Home Goal HT': h2h_match['score']['halfTime']['home'],
            'Away Goal HT': h2h_match['score']['halfTime']['away'],
            'Home Goal FT': h2h_match['score']['fullTime']['home'],
            'Away Goal FT': h2h_match['score']['fullTime']['away'],
        })

h2h_df = pd.DataFrame(h2h_list)


h2h_df.head()

,Match Id,H2H Match Id,Home Team,Away Team,Home Goal HT,Away Goal HT,Home Goal FT,Away Goal FT
0,497410,497410,Man United,Fulham,0,0,1,0
1,497410,436200,Man United,Fulham,0,0,1,2
2,497410,436046,Fulham,Man United,0,0,0,1
3,497410,416006,Man United,Fulham,1,1,2,1
4,497410,416232,Fulham,Man United,0,1,1,2


In [184]:
def assign_label(home_goal_ft, away_goal_ft):
    if home_goal_ft > away_goal_ft:
        return 1
    elif home_goal_ft < away_goal_ft:
        return -1
    else:
        return 0

match_df['Label'] = match_df.apply(lambda row: assign_label(row['Home Goal FT'], row['Away Goal FT']), axis=1)

match_df.head()

,Match Id,Home Team,Away Team,Home Goal HT,Away Goal HT,Home Goal FT,Away Goal FT,Label
0,497410,Man United,Fulham,0.0,0.0,1.0,0.0,1
1,497411,Ipswich Town,Liverpool,0.0,0.0,0.0,2.0,-1
2,497412,Arsenal,Wolverhampton,1.0,0.0,2.0,0.0,1
3,497413,Everton,Brighton Hove,0.0,1.0,0.0,3.0,-1
4,497414,Newcastle,Southampton,1.0,0.0,1.0,0.0,1


In [185]:
h2h_df['Label'] = h2h_df.apply(lambda row: assign_label(row['Home Goal FT'], row['Away Goal FT']), axis=1)

h2h_df.head()

,Match Id,H2H Match Id,Home Team,Away Team,Home Goal HT,Away Goal HT,Home Goal FT,Away Goal FT,Label
0,497410,497410,Man United,Fulham,0,0,1,0,1
1,497410,436200,Man United,Fulham,0,0,1,2,-1
2,497410,436046,Fulham,Man United,0,0,0,1,-1
3,497410,416006,Man United,Fulham,1,1,2,1,1
4,497410,416232,Fulham,Man United,0,1,1,2,-1


In [189]:
def form_to_points(form):
    points = 0
    for i, result in enumerate(form[::-1]):
        if result == 'W':
            points += 3 * (1.2 ** i)
        elif result == 'D':
            points += 1 * (1.2 ** i)
    return points

standing_df.loc[:, 'Form Score'] = standing_df['Form'].apply(form_to_points).round(4)

standing_df[['Team', 'Form', 'Form Score']].head()

,Team,Form,Form Score
0,Liverpool,"W,W,W,L,W",31.0782
1,Man City,"W,D,D,W,W",25.2790
2,Arsenal,"W,W,D,W,D",29.2510
3,Chelsea,"D,W,W,W,D",24.7986
4,Aston Villa,"D,D,W,W,W",20.8266


In [187]:
match_df = match_df.merge(standing_df[['Team', 'Pos', 'P', 'GD', 'Form Score']], 
                          left_on='Home Team', right_on='Team', how='left', suffixes=('', '_home'))
match_df = match_df.merge(standing_df[['Team', 'Pos', 'P', 'GD', 'Form Score']], 
                          left_on='Away Team', right_on='Team', how='left', suffixes=('_home', '_away'))

match_df.drop(['Team_home', 'Team_away'], axis=1, inplace=True)

match_df.head()

,Match Id,Home Team,Away Team,Home Goal HT,Away Goal HT,Home Goal FT,Away Goal FT,Label,Pos_home,P_home,GD_home,Form Score_home,Pos_away,P_away,GD_away,Form Score_away
0,497410,Man United,Fulham,0.0,0.0,1.0,0.0,1,14,8,-3,10.6934,8,11,2,17.6188
1,497411,Ipswich Town,Liverpool,0.0,0.0,0.0,2.0,-1,17,4,-8,7.4996,1,18,11,31.0782
2,497412,Arsenal,Wolverhampton,1.0,0.0,2.0,0.0,1,3,17,9,29.2510,20,1,-12,1.0000
3,497413,Everton,Brighton Hove,0.0,1.0,0.0,3.0,-1,16,5,-8,15.3314,6,12,3,17.4131
4,497414,Newcastle,Southampton,1.0,0.0,1.0,0.0,1,7,12,1,14.6058,19,1,-11,2.0736


In [192]:
h2h_df['H2H Point'] = h2h_df.apply(lambda x: 3 if x['Home Goal FT'] > x['Away Goal FT'] else (1 if x['Home Goal FT'] == x['Away Goal FT'] else 0), axis=1)

h2h_agg = h2h_df.groupby('Match Id').agg({
    'H2H Point': ['sum']
}).reset_index()

h2h_agg.columns = ['Match Id', 'H2H Points']

match_df = match_df.merge(h2h_agg, on='Match Id', how='left')

match_df.head()

,Match Id,Home Team,Away Team,Home Goal HT,Away Goal HT,Home Goal FT,Away Goal FT,Label,Pos_home,P_home,GD_home,Form Score_home,Pos_away,P_away,GD_away,Form Score_away,H2H Points_x,H2H Points_y,H2H Points
0,497410,Man United,Fulham,0.0,0.0,1.0,0.0,1,14,8,-3,10.6934,8,11,2,17.6188,6.0,6.0,6.0
1,497411,Ipswich Town,Liverpool,0.0,0.0,0.0,2.0,-1,17,4,-8,7.4996,1,18,11,31.0782,0.0,0.0,0.0
2,497412,Arsenal,Wolverhampton,1.0,0.0,2.0,0.0,1,3,17,9,29.2510,20,1,-12,1.0000,9.0,9.0,9.0
3,497413,Everton,Brighton Hove,0.0,1.0,0.0,3.0,-1,16,5,-8,15.3314,6,12,3,17.4131,2.0,2.0,2.0
4,497414,Newcastle,Southampton,1.0,0.0,1.0,0.0,1,7,12,1,14.6058,19,1,-11,2.0736,6.0,6.0,6.0
